# U-nets, vortex detection, and semantic segmentation

In this file, we will train a semantic image segmentation network to identify regions of high vorticity and velocity in our simulation, given only a screenshot of the fluid density. One appropriate choice of network architecture for this scenario is the U-net, named for its characteristic shape:

![](images/unet-arch.png)

The intuition behind the U-net is that convolutional implementations coarse-grain data in the input image in order to extract low-level feature data. Coarse-graining is a non-invertible process and thus destroys information about correlations, so we feed the data at each layer forward, to build up an image which has the same size and resolution as the input and has access to the correlations at each CONV block.

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, UpSampling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [2]:
def UNet(input_shape = (128, 128, 3)):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    #X = ZeroPadding2D((3, 3))(X_input)
    #X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    #X = AveragePooling2D(pool_size=(2,2), name='avg_pool')(X)
    #X = Flatten()(X)
    #X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Stage 1d
    X = Conv2D(64, (3, 3), strides = (1, 1), name = 'convd1a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convd1a')(X)
    X = Activation('relu')(X)
    X = Conv2D(64, (3, 3), strides = (1, 1), name = 'convd1b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convd1b')(X)
    X = Activation('relu')(X)
    X_shortcut_1 = X
    
    # Downward step 1
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)

    # Stage 2d
    X = Conv2D(128, (3, 3), strides = (1, 1), name = 'convd2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convd2a')(X)
    X = Activation('relu')(X)
    X = Conv2D(128, (3, 3), strides = (1, 1), name = 'convd2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convd2b')(X)
    X = Activation('relu')(X)
    X_shortcut_2 = X
    
    # Downward step 2d
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)

    # Stage 3d 
    X = Conv2D(256, (3, 3), strides = (1, 1), name = 'convd3a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convd3a')(X)
    X = Activation('relu')(X)
    X = Conv2D(256, (3, 3), strides = (1, 1), name = 'convd3b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convd3b')(X)
    X = Activation('relu')(X)
    X_shortcut_3 = X

    # Downward step 3
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    
    # Stage 4d
    X = Conv2D(512, (3, 3), strides = (1, 1), name = 'convd4a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convd4a')(X)
    X = Activation('relu')(X)
    X = Conv2D(512, (3, 3), strides = (1, 1), name = 'convd4b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convd4b')(X)
    X = Activation('relu')(X)
    X_shortcut_4 = X

    # Downward step 4
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    
    # Stage 5b
    X = Conv2D(1024, (3, 3), strides = (1, 1), name = 'convb5a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convb5a')(X)
    X = Activation('relu')(X)
    X = Conv2D(1024, (3, 3), strides = (1, 1), name = 'convb5b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convb5b')(X)
    X = Activation('relu')(X)
    
    # Upward step 4
    X = UpSampling2D(size=(2, 2), interpolation='nearest')(X)
    X = Add()([X_shortcut_4,X])
    
    # Stage 4u
    X = Conv2D(512, (3, 3), strides = (1, 1), name = 'convu4a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convu4a')(X)
    X = Activation('relu')(X)
    X = Conv2D(512, (3, 3), strides = (1, 1), name = 'convu4b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convu4b')(X)
    X = Activation('relu')(X)
    
    # Upward step 3
    X = UpSampling2D(size=(2, 2), interpolation='nearest')(X)
    X = Add()([X_shortcut_3,X])
    
    # Stage 3u
    X = Conv2D(256, (3, 3), strides = (1, 1), name = 'convu3a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convu3a')(X)
    X = Activation('relu')(X)
    X = Conv2D(256, (3, 3), strides = (1, 1), name = 'convu3b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convu3b')(X)
    X = Activation('relu')(X)
    
    # Upward step 2
    X = UpSampling2D(size=(2, 2), interpolation='nearest')(X)
    X = Add()([X_shortcut_2,X])
    
    # Stage 2u
    X = Conv2D(128, (3, 3), strides = (1, 1), name = 'convu2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convu2a')(X)
    X = Activation('relu')(X)
    X = Conv2D(128, (3, 3), strides = (1, 1), name = 'convu2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convu2b')(X)
    X = Activation('relu')(X)
    
    # Upward step 1
    X = UpSampling2D(size=(2, 2), interpolation='nearest')(X)
    X = Add()([X_shortcut_1,X])
    
    # Stage 1u
    X = Conv2D(64, (3, 3), strides = (1, 1), name = 'convu1a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convu1a')(X)
    X = Activation('relu')(X)
    X = Conv2D(64, (3, 3), strides = (1, 1), name = 'convu1b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convu1b')(X)
    X = Activation('relu')(X)
    
    # output layer
    X = Conv2D(1, (1, 1), strides = (1, 1), name = 'convOut', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_convOut')(X)
    X = Activation('sigmoid')(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='UNet')

    return model

In [3]:
model = UNet(input_shape = (128, 128, 3))

NameError: name 'ResNet50' is not defined

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
my_image = scipy.misc.imread(img_path)
imshow(my_image)
print("class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ")
print(model.predict(x))